### **Step 1: Install & Import Required Libraries**

In [1]:
!pip install mysql-connector-python pandas tqdm

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 4.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 5.0 MB/s eta 0:00:00a 0:00:01
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 3.0 MB/s eta 0:00:00a 0:00:01
Using cached pytz-2025.1-py2.py3-none-any.whl (507 kB)
Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)


In [2]:
import mysql.connector
import pandas as pd
import logging
from tqdm import tqdm

### **Step 2: Define Database Connection**

In [6]:
# Database connection details
DB_CONFIG = {
    'host': '34.46.229.160',
    'user': 'root',
    'password': 'b)p[/&GH-o|B]6u+',
    'database': 'loan_data',
    'table_name': 'credit_record_final'
}

# Initialize Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

### **Step 3: Load `credit_record.csv` into Pandas DataFrame**

In [5]:
# Define file path
csv_file_path = 'credit_card_data/credit_record.csv'

# Load CSV into DataFrame
df = pd.read_csv(csv_file_path)

# Show first few rows
df.head()

ID  MONTHS_BALANCE STATUS
0  5001711               0      X
1  5001711              -1      0
2  5001711              -2      0
3  5001711              -3      0
4  5001712               0      C

### **Step 4: Connect to MySQL and Create Table if Not Exists**

In [7]:
try:
    # Connect to MySQL database
    conn = mysql.connector.connect(
        host=DB_CONFIG['host'],
        user=DB_CONFIG['user'],
        password=DB_CONFIG['password'],
        database=DB_CONFIG['database']
    )
    cursor = conn.cursor()
    
    # Create table if not exists
    create_table_query = f'''
    CREATE TABLE IF NOT EXISTS {DB_CONFIG['table_name']} (
        ID VARCHAR(50),
        MONTHS_BALANCE INT,
        STATUS VARCHAR(10),
        PRIMARY KEY (ID, MONTHS_BALANCE)
    ) ENGINE=InnoDB;
    '''
    cursor.execute(create_table_query)
    conn.commit()
    logging.info("Table `credit_record` created successfully (if not already exists).")
    
except Exception as e:
    logging.error(f"Error creating table: {e}")
finally:
    cursor.close()
    conn.close()

2025-03-21 22:11:21,441 - INFO - Table `credit_record` created successfully (if not already exists).


### **Step 5: Upload Data to MySQL in Batches**

In [10]:
import mysql.connector
import logging
from tqdm import tqdm


try:
    # Reconnect to MySQL
    conn = mysql.connector.connect(
        host=DB_CONFIG['host'],
        user=DB_CONFIG['user'],
        password=DB_CONFIG['password'],
        database=DB_CONFIG['database']
    )
    cursor = conn.cursor()
    
    # Insert Data in Batches
    batch_size = 1000
    insert_query = f"""INSERT INTO {DB_CONFIG['table_name']} (ID, MONTHS_BALANCE, STATUS) 
                      VALUES (%s, %s, %s) 
                      ON DUPLICATE KEY UPDATE STATUS=VALUES(STATUS)"""  # Avoid duplicates
    
    records = df.to_records(index=False)
    records_list = [(str(record[0]), int(record[1]), str(record[2])) for record in records]  # Convert records to tuples with native Python types
    
    for i in tqdm(range(0, len(records_list), batch_size)):
        batch = records_list[i:i+batch_size]
        cursor.executemany(insert_query, batch)
        conn.commit()
        logging.info(f"Inserted {i + len(batch)} records so far...")
    
    logging.info("Data upload completed successfully!")

except mysql.connector.Error as e:
    logging.error(f"MySQL error: {e}")
except Exception as e:
    logging.error(f"Error inserting data: {e}")
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()

100%|██████████| 1049/1049 [10:45<00:00,  1.62it/s]
2025-03-21 22:26:39,160 - INFO - Data upload completed successfully!


### **Step 6: Verify Data Upload in MySQL**

In [11]:
try:
    # Reconnect to MySQL
    conn = mysql.connector.connect(
        host=DB_CONFIG['host'],
        user=DB_CONFIG['user'],
        password=DB_CONFIG['password'],
        database=DB_CONFIG['database']
    )
    cursor = conn.cursor()
    
    # Count Records
    cursor.execute(f"SELECT COUNT(*) FROM {DB_CONFIG['table_name']}")
    record_count = cursor.fetchone()[0]
    logging.info(f"Total records in `{DB_CONFIG['table_name']}`: {record_count}")
    
except Exception as e:
    logging.error(f"Error verifying data: {e}")
finally:
    cursor.close()
    conn.close()

2025-03-21 22:33:18,693 - INFO - Total records in `credit_record_final`: 1048575
